In [42]:
import pandas as pd

datapath = 'ibu.csv' #Tüm dil verilerin bulunduğu csv dosyasının yolu 
df = pd.read_csv(datapath, delimiter="|") 
df.head()

,id,text,is_offensive,target
0,d93e05f7-bfdd-4cdb-99d8-3048761b30ff,otomasyon sistemlerine doğrudan bağlanabilir,0,OTHER
1,2dd1b0d1-45cd-4d4a-86c3-b290db5a0316,makara bunu duyunca avluya çıkmış ve kızağa sa...,0,OTHER
2,c5291181-1411-4cec-aa06-1a279bc8f45e,ben bunu istemedim,0,OTHER
3,f2c118d4-50e3-4346-9473-e7f017b674b7,ancak bana geri dönmediler,0,OTHER
4,2a63033c-2526-4c71-a9f5-da867e8e73b4,özellikle büyük şehirlere aynı gün içinde deği...,0,OTHER


In [2]:
from transformers import BertTokenizer  # transformers kütüphanesinden BertTokenizer import ediliyor

tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')  # BertTokenizer modeli yükleniyor

example_text = 'I will watch Memento tonight'  # örnek metin tanımlanıyor

bert_input = tokenizer(example_text, padding='max_length', max_length=20,  # tokenizer ile örnek metin dönüştürülüyor
                       truncation=True, return_tensors="pt")

print(bert_input['input_ids'])  # input_ids dizisi yazdırılıyor
print(bert_input['token_type_ids'])  # token_type_ids dizisi yazdırılıyor
print(bert_input['attention_mask'])  # attention_mask dizisi yazdırılıyor

tensor([[  101,   146, 11337, 34481, 68360, 22464, 27521,   102,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0]])
tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])


In [ ]:
import numpy as np

In [5]:
example_text = tokenizer.decode(bert_input.input_ids[0])  # model tarafından kodlanan girdi dizisi geri dönüştürülüyor

print(example_text)

[CLS] I will watch Memento tonight [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]


In [1]:
import torch
import numpy as np
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')  # BertTokenizer modeli yükleniyor

labels = {'OTHER':0,  # etiketler tanımlanıyor
          'INSULT':1,
          'PROFANITY':2,
          'RACIST':3,
          'SEXIST':4
          }

class Dataset(torch.utils.data.Dataset):  # Dataset sınıfı tanımlanıyor

    def __init__(self, df):  # sınıfın özellikleri tanımlanıyor

        self.labels = [labels[label] for label in df['target']]  # etiketler özellik olarak saklanıyor
        self.texts = [tokenizer(text,  # metinler tokenizer ile kodlanıyor
                               padding='max_length', max_length=512, truncation=True,
                                return_tensors="pt") for text in df['text']]

    def classes(self):  # etiketlerin listesini döndüren fonksiyon tanımlanıyor
        return self.labels

    def __len__(self):  # örnek sayısı döndüren fonksiyon tanımlanıyor
        return len(self.labels)

    def get_batch_labels(self, idx):  # belirtilen indisteki etiketleri döndüren fonksiyon tanımlanıyor
        return np.array(self.labels[idx])

    def get_batch_texts(self, idx):  # belirtilen indisteki metinleri döndüren fonksiyon tanımlanıyor
        return self.texts[idx]

    def __getitem__(self, idx):  # belirtilen indisteki örnekleri döndüren fonksiyon tanımlanıyor

        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_labels(idx)

        return batch_texts, batch_y

c:\Users\alize\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [43]:
np.random.seed(112)  # rastgelelik için seed belirleniyor

# Veri kümesi bölünüyor ve eğitim, doğrulama ve test setleri olarak ayrılıyor
df_train, df_val, df_test = np.split(df.sample(frac=1, random_state=42), 
                                     [int(.8*len(df)), int(.9*len(df))])

print(len(df_train),len(df_val), len(df_test))  # set boyutları yazdırılıyor

9928 1241 1242


In [48]:
df_train["target"].value_counts()

OTHER        2840
INSULT       1930
PROFANITY    1883
SEXIST       1668
RACIST       1607
Name: target, dtype: int64

In [2]:
from torch import nn
from transformers import BertModel

class BertClassifier(nn.Module):

    def __init__(self, dropout=0.5):

        super(BertClassifier, self).__init__()

        # Önceden eğitilmiş BERT modeli yükleniyor
        self.bert = BertModel.from_pretrained('bert-base-multilingual-cased')
        
        self.dropout = nn.Dropout(dropout)  # Dropout katmanı tanımlanıyor
        self.linear = nn.Linear(768, 5)  # Çıkış katmanı tanımlanıyor
        self.relu = nn.ReLU()  # Aktivasyon fonksiyonu tanımlanıyor

    def forward(self, input_id, mask):
        
        # BERT modeline girdi veriliyor ve çıktı elde ediliyor
        _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)
        
        # Dropout uygulanıyor
        dropout_output = self.dropout(pooled_output)
        
        # Çıkış katmanına veri veriliyor
        linear_output = self.linear(dropout_output)
        
        # Aktivasyon fonksiyonu uygulanıyor
        final_layer = self.relu(linear_output)

        return final_layer

In [25]:
def train(model, train_data, val_data, learning_rate, epochs):
    # Eğitim ve doğrulama veri setlerini oluştur
    train, val = Dataset(train_data), Dataset(val_data)

    # Eğitim ve doğrulama için veri yükleyicilerini tanımla
    train_dataloader = torch.utils.data.DataLoader(train, batch_size=2, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val, batch_size=2)

    # GPU kullanılabilirliğini kontrol et
    use_cuda = torch.cuda.is_available()
    # Kullanılabilirse CUDA cihazını kullan, değilse CPU'yu kullan
    device = torch.device("cuda" if use_cuda else "cpu")

    # Kayıp fonksiyonunu tanımla
    criterion = nn.CrossEntropyLoss()
    # Optimizasyon algoritmasını tanımla
    optimizer = Adam(model.parameters(), lr= learning_rate)

    # Eğer GPU kullanılıyorsa, modeli GPU'ya taşı
    if use_cuda:
        model = model.cuda()
        criterion = criterion.cuda()

    # Belirtilen sayıda epoch için eğitim döngüsü

    for epoch_num in range(epochs):

            total_acc_train = 0
            total_loss_train = 0

            for train_input, train_label in tqdm(train_dataloader):

                train_label = train_label.to(device)
                mask = train_input['attention_mask'].to(device)
                input_id = train_input['input_ids'].squeeze(1).to(device)

                output = model(input_id, mask)
                
                batch_loss = criterion(output, train_label.long())
                total_loss_train += batch_loss.item()
                
                acc = (output.argmax(dim=1) == train_label).sum().item()
                total_acc_train += acc
                # Ağırlıkların sıfırlanması, türevlerin hesaplanması ve geri yayılım
                model.zero_grad()
                batch_loss.backward()
                optimizer.step()
            # Doğrulama verileri üzerinde doğruluk ve kayıp hesaplanır
            total_acc_val = 0
            total_loss_val = 0

            with torch.no_grad():

                for val_input, val_label in val_dataloader:

                    val_label = val_label.to(device)
                    mask = val_input['attention_mask'].to(device)
                    input_id = val_input['input_ids'].squeeze(1).to(device)

                    output = model(input_id, mask)

                    batch_loss = criterion(output, val_label.long())
                    total_loss_val += batch_loss.item()
                    
                    acc = (output.argmax(dim=1) == val_label).sum().item()
                    total_acc_val += acc
            # Her epoch'tan sonra eğitim ve doğrulama doğruluğu ve kaybı yazdırılır
            print(
                f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_data): .3f} \
                | Train Accuracy: {total_acc_train / len(train_data): .3f} \
                | Val Loss: {total_loss_val / len(val_data): .3f} \
                | Val Accuracy: {total_acc_val / len(val_data): .3f}')
                  
EPOCHS = 5
model = BertClassifier()
LR = 1e-6
              
train(model, df_train, df_val, LR, EPOCHS)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
 12%|█▏        | 579/4964 [02:37<19:53,  3.67it/s]


KeyboardInterrupt: 

In [13]:
# Evaluates a PyTorch model on a test dataset
# by computing the accuracy of the model's predictions

model.eval()  # set model to evaluation mode

def evaluate(model, test_data):

    test = Dataset(test_data)  # create test dataset

    test_dataloader = torch.utils.data.DataLoader(test, batch_size=2)  # create test dataloader

    use_cuda = torch.cuda.is_available()  # check if GPU is available
    device = torch.device("cuda" if use_cuda else "cpu")  # set device based on availability of GPU

    if use_cuda:
        model = model.cuda()  # move model to GPU if available

    total_acc_test = 0  # initialize total accuracy to 0
    with torch.no_grad():  # don't compute gradients during evaluation
        for test_input, test_label in test_dataloader:
            test_label = test_label.to(device)  # move test labels to GPU if available
            mask = test_input['attention_mask'].to(device)  # move attention mask to GPU if available
            input_id = test_input['input_ids'].squeeze(1).to(device)  # move input IDs to GPU if available
            output = model(input_id, mask)  # get model's predictions for the current batch
            acc = (output.argmax(dim=1) == test_label).sum().item()  # compute accuracy for the current batch
            total_acc_test += acc  # add current batch accuracy to total accuracy

    print(f'Test Accuracy: {total_acc_test / len(test_data): .3f}')  # print final test accuracy
    
evaluate(model, df_test)  # evaluate model on test dataset

NameError: name 'df_test' is not defined

INFERENCE

INFERENCE

In [8]:
import torch
import pandas as pd
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

from torch import nn
from transformers import BertModel

class BertClassifier(nn.Module):
# BertClassifier adında bir sınıf tanımlanıyor ve nn.Module sınıfından kalıtım alıyor.

    def __init__(self, dropout=0.5):
    # Sınıfın özellikleri (attributes) burada tanımlanıyor. Dropout default olarak 0.5 olarak belirleniyor.

        super(BertClassifier, self).__init__()
        # BertClassifier sınıfı nn.Module sınıfından kalıtım aldığı için nn.Module sınıfının __init__() metodunu da çağırıyor.

        self.bert = BertModel.from_pretrained('bert-base-multilingual-cased')
        # bert-base-multilingual-cased modeli kullanılarak bir bert modeli oluşturuluyor.

        self.dropout = nn.Dropout(dropout)
        # dropout oranını alarak bir dropout katmanı oluşturuyor.

        self.linear = nn.Linear(768, 5)
        # Tam bağlı bir katman oluşturuluyor ve girdi boyutu 768, çıktı boyutu 5 olarak belirleniyor.

        self.relu = nn.ReLU()
        # ReLU aktivasyon fonksiyonu belirleniyor.

    def forward(self, input_id, mask):
    # İleri besleme (forward) işlemi burada tanımlanıyor.

        _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)
        # Bert modeli kullanılarak tokenlerin embedding'leri ve pooled_output vektörü elde ediliyor. Underscore (_) burada kullanılmış
        # ancak değeri kullanılmayacağı için atıyoruz.

        dropout_output = self.dropout(pooled_output)
        # Dropout uygulanıyor.

        linear_output = self.linear(dropout_output)
        # Tam bağlı katman uygulanıyor.

        final_layer = self.relu(linear_output)
        # ReLU aktivasyon fonksiyonu uygulanıyor.

        return final_layer
        # Son katmanın çıktısı döndürülüyor.


model = torch.load("model.pt")

model = model.eval()

model = model.cuda()

def inference(ex_text, target):

    # ex_text ve target'ı numpy dizisi olarak tanımla
    txt = np.array([ex_text])
    target = np.array([target])

    # sütun değerlerini belirle
    column_values = ['text', 'target']

    # inferencedf adlı veri çerçevesini oluştur
    inferencedf = pd.DataFrame(data = np.column_stack([txt, target]), columns = column_values)

    # inference adlı veri kümesini oluştur
    inference = Dataset(inferencedf)

    # inference_dataloader'ı tanımla
    inference_dataloader = torch.utils.data.DataLoader(inference, batch_size=1, shuffle=False)

    # hesaplamaları GPU'ya gönder
    with torch.no_grad():
        # batch'e göre inferencedataloader'dan verileri al
        for batch in inference_dataloader:
            # maskelemeyi cihaza gönder
            mask = batch[0]['attention_mask'].to(device)
            # girdi kimliklerini (input ids) cihaza gönder
            input_id = batch[0]['input_ids'].squeeze(1).to(device)
            # model tahminini cihazda hesapla
            output = model(input_id, mask).to(device)
            # tahminlerin ilk elemanını yazdır
            print(output[0].tolist())

    # modelin çıktısının bir liste şeklinde a'ya atanması
    a = output[0].tolist()

    # maksimum değeri döndüren index'in get_key fonksiyonuna gönderilmesi
    a = a.index(max(a))

    return get_key(a)

# etiket sözlüğünün tanımlanması
labeldict = {'OTHER':0,
             'INSULT':1,
             'PROFANITY':2,
             'RACIST':3,
             'SEXIST':4
            }

def get_key(val):
    for key, value in labeldict.items():
        if val == value:
            return key


In [9]:
ex_text = "okula geri gitmek gerçekten insanı üzüyor "
target = "OTHER"
        
print(inference(ex_text))

TypeError: inference() missing 1 required positional argument: 'target'

CLEANER INFERENCE

In [ ]:
import torch
import pandas as pd
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

from torch import nn
from transformers import BertModel, BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
labels = {'OTHER':0,
          'INSULT':1,
          'PROFANITY':2,
          'RACIST':3,
          'SEXIST':4
          }

class Dataset(torch.utils.data.Dataset):

    def __init__(self, df):

        # Girdi dataframe'inden metinlerin tokenleştirilmiş hallerini çıkar
        self.texts = [tokenizer(text, 
                               padding='max_length', max_length=512, truncation=True,
                                return_tensors="pt") for text in df['text']]

    def __len__(self):
        # Dataset'in boyutunu döndürür
        return len(self.texts)

    def get_batch_texts(self, idx):
        # idx numaralı girdi yığınını döndürür
        return self.texts[idx]

    def __getitem__(self, idx):
        # idx numaralı girdi yığınına karşılık gelen öğeyi döndürür
        batch_texts = self.get_batch_texts(idx)
        return batch_texts

    
class BertClassifier(nn.Module):

    def __init__(self, dropout=0.5):

        super(BertClassifier, self).__init__()

        self.bert = BertModel.from_pretrained('bert-base-multilingual-cased')
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 5)
        self.relu = nn.ReLU()

    def forward(self, input_id, mask):

        _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.relu(linear_output)

        return final_layer


model = torch.load("model.pt")

model = model.eval()

model = model.cuda()

def inference(ex_text):

    txt = np.array([ex_text])
    #target = np.array([target])
    column_values = ['text'] #    column_values = ['text', 'target']

    inferencedf = pd.DataFrame(data = np.column_stack([txt]), columns = column_values)
    #inferencedf = pd.DataFrame(data=np.column_stack([txt[:, None]]), columns=column_values)

    #inferencedf = pd.DataFrame(data = np.column_stack([txt, target]), columns = column_values)
    #inferencedf = pd.read_csv("testpdf.csv", index_col=0)
    inference = Dataset(inferencedf)

    inference_dataloader = torch.utils.data.DataLoader(inference, batch_size=1, shuffle=False)

    with torch.no_grad():
        for batch in inference_dataloader:
            mask = batch[0]['attention_mask'].to(device)
            input_id = batch[0]['input_ids'].squeeze(1).to(device)
            output = model(input_id, mask).to(device)
            print(output[0].tolist())

    a = output[0].tolist()

    a = np.argmax(a)
    
    return get_key(a)
    
labeldict = {'OTHER':0,
                 'INSULT':1,
                 'PROFANITY':2,
                 'RACIST':3,
                 'SEXIST':4
              }

def get_key(val):
    for key, value in labeldict.items():
        if val == value:
            return key
        
ex_text = "okula geri gitmek gerçekten insanı üzüyor "
target = "OTHER"
        
print(inference(ex_text))


CHATGPT HELP - SOLVED

TEXT

In [3]:
import torch
import pandas as pd
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

from torch import nn
from transformers import BertModel, BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
labels = {'OTHER':0,
          'INSULT':1,
          'PROFANITY':2,
          'RACIST':3,
          'SEXIST':4
          }

class Dataset(torch.utils.data.Dataset):

    def __init__(self, df):

        self.texts = [tokenizer(text, 
                               padding='max_length', max_length = 512, truncation=True,
                                return_tensors="pt") for text in df['text']]

    def __len__(self):
        return len(self.texts)

    def get_batch_texts(self, idx):
        # Fetch a batch of inputs
        return self.texts[idx]

    def __getitem__(self, idx):

        batch_texts = self.get_batch_texts(idx)
        return batch_texts
    
class BertClassifier(nn.Module):

    def __init__(self, dropout=0.5):

        super(BertClassifier, self).__init__()

        self.bert = BertModel.from_pretrained('bert-base-multilingual-cased')
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 5)
        self.relu = nn.ReLU()

    def forward(self, input_id, mask):

        _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.relu(linear_output)

        return final_layer


model = torch.load("model.pt")  # Eğitilmiş modelin diskten yüklenmesi
model = model.eval()  # Modelin test edilmek üzere ayarlanması
model = model.cuda()  # Modelin CUDA desteği ile kullanılabilmesi için GPU'ya yüklenmesi

def inference(ex_text):
    txt = np.array([ex_text])  # Girdi metninin numpy dizisi haline getirilmesi
    column_values = ['text']
    inferencedf = pd.DataFrame(data = np.column_stack([txt]), columns = column_values)  # Girdi verisinin pandas DataFrame'ine dönüştürülmesi
    inference = Dataset(inferencedf)  # Test seti olarak kullanılmak üzere girdi verisinin yüklenmesi
    inference_dataloader = torch.utils.data.DataLoader(inference, batch_size=1, shuffle=False)  # DataLoader'ın oluşturulması

    with torch.no_grad():
        for batch in inference_dataloader:
            mask = batch['attention_mask'].squeeze(0).to(device)  # Dikkat maskesi tensörünün oluşturulması ve GPU'ya yüklenmesi
            input_id = batch['input_ids'].squeeze(0).to(device)  # Girdi tensörünün oluşturulması ve GPU'ya yüklenmesi
            output = model(input_id, mask).to(device)  # Modelin girdi tensörüne dayanarak çıktı tensörünün oluşturulması ve GPU'ya yüklenmesi
            print(output[0].tolist())  # Çıktı tensörünün listeye dönüştürülmesi ve yazdırılması

    a = output[0].tolist()  # Çıktı tensörünün listeye dönüştürülmesi
    a = np.argmax(a)  # Çıktı tensöründeki en yüksek değere sahip indeksin elde edilmesi
    return get_key(a)  # Etiket indeksine karşılık gelen etiketin get_key fonksiyonu ile elde edilmesi ve döndürülmesi


    
labeldict = {'OTHER':0,
                 'INSULT':1,
                 'PROFANITY':2,
                 'RACIST':3,
                 'SEXIST':4
              }

def get_key(val):
    for key, value in labeldict.items():
        if val == value:
            return key
        
ex_text = "kötü ibne "
target = "OTHER"
        
print(inference(ex_text))


[0.0, 0.0, 6.152346134185791, 0.0, 0.0]
PROFANITY


DATAFRAME

In [24]:
import torch
import pandas as pd
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

from torch import nn
from transformers import BertModel, BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

class Dataset(torch.utils.data.Dataset):

    def __init__(self, df):

        self.texts = [tokenizer(text, 
                               padding='max_length', max_length = 512, truncation=True,
                                return_tensors="pt") for text in df['text']]

    def __len__(self):
        return len(self.texts)

    def get_batch_texts(self, idx):
        # Fetch a batch of inputs
        return self.texts[idx]

    def __getitem__(self, idx):

        batch_texts = self.get_batch_texts(idx)
        return batch_texts
    
class BertClassifier(nn.Module):

    def __init__(self, dropout=0.5):

        super(BertClassifier, self).__init__()

        self.bert = BertModel.from_pretrained('bert-base-multilingual-cased')
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 5)
        self.relu = nn.ReLU()

    def forward(self, input_id, mask):

        _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.relu(linear_output)

        return final_layer
    

model = torch.load("model.pt")

model = model.eval()

model = model.cuda()

def inference(df):

    output_list = []

    inference = Dataset(df)
    inference_dataloader = torch.utils.data.DataLoader(inference, batch_size=1, shuffle=False)

    with torch.no_grad():
        for batch in inference_dataloader:
            mask = batch['attention_mask'].squeeze(0).to(device)
            input_id = batch['input_ids'].squeeze(0).to(device)
            output = model(input_id, mask).to(device)
            output_list.append(output[0].tolist())


    #a = output[0].tolist()
    #a = np.argmax(a)
    return output_list
    

def get_key(val):
    # iterate over labeldict to find the key for the given value
    for key, value in labeldict.items():
        if val == value:
            return key
        
# define a dictionary of label values and their corresponding keys
labeldict = {'OTHER':0,
                 'INSULT':1,
                 'PROFANITY':2,
                 'RACIST':3,
                 'SEXIST':4
              }

# read the test data from a csv file using pandas
df = pd.read_csv("testpdf.csv", delimiter="|")

# call the inference function on the test data and print the results
print(inference(df))

[[4.343591213226318, 1.3162342309951782, 0.906980037689209, 0.0, 0.0], [0.0, 5.220179557800293, 0.0, 0.0, 0.0], [4.8646721839904785, 0.33086317777633667, 0.0, 0.0, 0.0]]
